# resent test model

In [ ]:
# Define paths
cropped_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test/test_cropped/cropped"
output_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/output/"

train_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/train/train"
test_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test"

validated_images_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test_cropped/validated"

processed_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/processed/"
error_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/error/"

class_labels_file = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/class_labels.pkl"
model_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/resnet_model.h5"

In [3]:
# Import libraries
import os
import cv2
import numpy as np
from keras.models import load_model
import pickle
import shutil
import time
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

# Define paths
cropped_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test_cropped/"
output_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/output/"

train_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/train/train"
test_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test"

validated_images_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test_cropped/validated"

processed_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/processed/"
error_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/error/"

class_labels_file = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/class_labels.pkl"
model_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/resnet_model.h5"

# Define target size for images
target_image_size = (100, 100)

# Load class labels
with open(class_labels_file, "rb") as f:
    index_to_label = pickle.load(f)

# Function to resize an image
def resize_image(image, target_width=100, target_height=100):
    resized_image = cv2.resize(image, (target_width, target_height))
    return resized_image

# Function to preprocess an image
def preprocess_image(image):
    try:
        img = resize_image(image)  # Resize the image to target size
        img = img / 255.0  # Normalize pixel values
        return img
    except Exception as e:
        print(f"Error preprocessing image: {e}")
        return None

# Function to crop faces from images
def crop_save_image(image_path, output_folder):
    image = cv2.imread(image_path)
    if image is not None:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        # Process each detected face
        for i, (x, y, w, h) in enumerate(faces, 1):
            # Draw rectangle around the detected face
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            # Crop and save the face
            face = image[y:y+h, x:x+w]
            output_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_{i}_face.jpg")
            cv2.imwrite(output_path, face)

# Function to predict the class of an image
def predict_image_class(model, image): 
    try:
        preprocessed_img = np.expand_dims(image, axis=0)  # Add batch dimension
        prediction = model.predict(preprocessed_img)
        predicted_class_index = np.argmax(prediction)
        confidence_score = prediction[0][predicted_class_index]
        
        print("Index to label dictionary:", index_to_label)
        print("Predicted class index:", predicted_class_index)
        
        predicted_class_label = index_to_label.get(predicted_class_index)
        
        print(f"Predicted class label: {predicted_class_label}")
        print(f"Confidence score: {confidence_score}")
        
        return predicted_class_index, confidence_score  # Return index and confidence score
    
    except Exception as e:
        print(f"Error predicting image class: {e}")
        return None, None

# Function to test the model
def test_model(model_path, cropped_folder, output_folder, processed_folder, error_folder, index_to_label):
    
    print("Testing model...")
    
    try:
        start_time = time.time()
        print("Execution started...")

        # Load the trained model
        model = load_model(model_path)
    
        # Create output directories if they don't exist
        os.makedirs(output_folder, exist_ok=True)
        os.makedirs(processed_folder, exist_ok=True)
        os.makedirs(error_folder, exist_ok=True)

        # Crop and preprocess images for testing
        for filename in os.listdir(test_data_path):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                image_path = os.path.join(test_data_path, filename)
                crop_save_image(image_path, cropped_folder)

        # Process cropped images for testing
        for filename in os.listdir(cropped_folder):
            print(f"Processing image: {filename}")
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                image_path = os.path.join(cropped_folder, filename)
                image = cv2.imread(image_path)
                if image is not None:
                    image = preprocess_image(image)
                    if image is not None:
                        predicted_class_index, confidence_score = predict_image_class(model, image)
                        if predicted_class_index is not None:
                            predicted_class_label = index_to_label.get(predicted_class_index)
                            if predicted_class_label is not None:
                                person_folder = os.path.join(processed_folder, predicted_class_label)
                                os.makedirs(person_folder, exist_ok=True)
                                shutil.move(image_path, os.path.join(person_folder, filename))
                                print(f"Predicted: {predicted_class_label} with confidence: {confidence_score:.4f} - Image: {filename}")
                            else:
                                print(f"Error: Predicted class label not found for index {predicted_class_index}")
                                shutil.move(image_path, os.path.join(error_folder, filename))
                        else:
                            print(f"Error: Unable to predict class for image: {filename}")
                            shutil.move(image_path, os.path.join(error_folder, filename))
                    else:
                        print(f"Error: Preprocessing failed for image: {filename}")
                        shutil.move(image_path, os.path.join(error_folder, filename))
                else:
                    print(f"Error: Unable to read image: {filename}")
                    shutil.move(image_path, os.path.join(error_folder, filename))
        
        end_time = time.time()
        print("Execution completed.")
        print("Total execution time: {:.2f} seconds".format(end_time - start_time))
        
    except Exception as e:
        print(f"Error testing model: {e}")

# Test the model
test_model(model_path, cropped_folder, output_folder, processed_folder, error_folder, index_to_label)


Testing model...
Execution started...
Processing image: a1_1_face.jpg
1/1 [==============================] - 0s 473ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.9991785883903503
Error: Predicted class label not found for index 0
Processing image: a2_1_face.jpg
1/1 [==============================] - 0s 32ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.995481014251709
Error: Predicted class label not found for index 0
Processing image: ab1_1_face.jpg
1/1 [==============================] - 0s 39ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.5707178711891174
Error: Predicted class label not found for index 0
Proc

1/1 [==============================] - 0s 34ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.8485825657844543
Error: Predicted class label not found for index 0
Processing image: IMG_1896_1_face.jpg
1/1 [==============================] - 0s 30ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.9981170892715454
Error: Predicted class label not found for index 0
Processing image: IMG_1896_2_face.jpg
1/1 [==============================] - 0s 28ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.5522093772888184
Error: Predicted class label not found for index 0
Processing image: IMG_1896_3_face.jpg
1/1 [====================

1/1 [==============================] - 0s 36ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.6352181434631348
Error: Predicted class label not found for index 0
Processing image: IMG_4227_4_face.jpg
1/1 [==============================] - 0s 28ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.4127461910247803
Error: Predicted class label not found for index 0
Processing image: IMG_4229_1_face.jpg
1/1 [==============================] - 0s 29ms/step
Index to label dictionary: {'Aishwarya_Rai': 0, 'Latha NV': 1, 'Meenal': 2, 'Salman_Khan': 3}
Predicted class index: 0
Predicted class label: None
Confidence score: 0.9341661334037781
Error: Predicted class label not found for index 0
Processing image: IMG_4229_2_face.jpg
1/1 [====================

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Paths to test and train datasets
test_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/test/"
train_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/train/"

# Function to load and display an image with marked faces
def load_and_display_image_with_faces(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Load pre-trained face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Display the image with marked faces
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Load and display a test image with marked faces
test_image_path = os.path.join(test_data_path, "IMG_2014.jpg")
print("Test Image with Marked Faces:")
load_and_display_image_with_faces(test_image_path)


In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
import pickle
import matplotlib.pyplot as plt

# Paths to test and train datasets
test_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/test/"
train_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/train/"

# Load the trained model and class labels
model_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/trained_models/resnet_model.h5"
class_labels_file = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/trained_models/class_labels.pkl"

# Load class labels
with open(class_labels_file, "rb") as f:
    index_to_label = pickle.load(f)

# Function to crop faces marked with squares and predict their classes
def crop_faces_and_predict(image_path, model, index_to_label):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Load pre-trained face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5, minSize=(30, 30))
    
    for i, (x, y, w, h) in enumerate(faces, 1):
        # Crop the face region
        face = image[y:y+h, x:x+w]
        
        # Preprocess the cropped face for prediction
        preprocessed_face = cv2.resize(face, (100, 100)) / 255.0
        cv2.rectangle(preprocessed_face, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Display the image with marked faces
        plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
        # Expand dimensions to match the input shape of the model
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)
        
        # Predict the class of the cropped face
        prediction = model.predict(preprocessed_face)
        predicted_class_index = np.argmax(prediction)
        confidence_score = prediction[0][predicted_class_index]
        # predicted_class_label = index_to_label[predicted_class_index]
        print("index", index_to_label)
        
        #print("===============================================================")
        # Print prediction result
        # print(f"Predicted class label: {predicted_class_label}")
        print(f"Confidence score: {confidence_score}, Index: {predicted_class_index}")
        

# Load the trained model
model = load_model(model_path)

# Test the function with a test image
test_image_path = os.path.join(test_data_path, "IMG123.jpg")
crop_faces_and_predict(test_image_path, model, index_to_label)


In [ ]:
import os
import cv2
import numpy as np

# Paths to test and train datasets
test_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/test/"
train_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/train/"

# Function to encode images
def encode_images(data_path):
    encoded_images = []
    image_names = []

    for filename in os.listdir(data_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(data_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                # Encode the image
                encoded_image = cv2.imencode('.jpg', image)[1]
                encoded_images.append(encoded_image)
                image_names.append(filename)
    
    return encoded_images, image_names

# Encode images in train dataset
train_encoded_images, train_image_names = encode_images(train_data_path)

# Encode images in test dataset
test_encoded_images, test_image_names = encode_images(test_data_path)

# Print the encodings for train dataset
print("Train dataset encodings:")
for i, (encoded_image, image_name) in enumerate(zip(train_encoded_images, train_image_names), 1):
    print(f"Image {i}: {encoded_image}")
    print(f"Image Name: {image_name}")
    print()

# Print the encodings for test dataset
print("Test dataset encodings:")
for i, (encoded_image, image_name) in enumerate(zip(test_encoded_images, test_image_names), 1):
    print(f"Image {i}: {encoded_image}")
    print(f"Image Name: {image_name}")
    print()


In [5]:
import os
import cv2
import numpy as np
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
import pickle

# Define paths
train_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/train/"

# Load pre-trained ResNet model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add GlobalAveragePooling2D layer to extract features
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
feature_extractor = Model(inputs=resnet_model.input, outputs=x)

# Function to preprocess and extract features from images using ResNet model
def extract_features(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        # Resize image to fit ResNet input size
        resized_image = cv2.resize(image, (224, 224))
        # Preprocess input according to ResNet requirements
        preprocessed_image = preprocess_input(np.expand_dims(resized_image, axis=0))
        # Extract features using ResNet model
        features = feature_extractor.predict(preprocessed_image)
        return features.flatten()  # Flatten the features
    else:
        return None

# Function to encode images in the train dataset using ResNet model
def encode_images(data_path):
    encodings = []
    labels = []

    for label in os.listdir(data_path):
        label_folder = os.path.join(data_path, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                    image_path = os.path.join(label_folder, filename)
                    features = extract_features(image_path)
                    if features is not None:
                        encodings.append(features)
                        labels.append(label)
    
    return encodings, labels

# Encode images in the train dataset using ResNet model
train_encodings, train_labels = encode_images(train_data_path)

# Save the encodings and labels for future use
np.save("C:/Users/Learning/udemy/Hackathon/ultimate/dataset/trained_models/train_encodings.npy", train_encodings)
np.save("C:/Users/Learning/udemy/Hackathon/ultimate/dataset/trained_models/train_labels.npy", train_labels)


1/1 [==============================] - 0s 323ms/step


1/1 [==============================] - 0s 140ms/step


1/1 [==============================] - 0s 172ms/step


In [15]:
import os
import cv2
import numpy as np
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
import pickle

class_labels_file = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/trained_models/class_labels.pkl"

# Load class labels
with open(class_labels_file, "rb") as f:
    index_to_label = pickle.load(f)

# Load the trained model
model = load_model("C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\trained_models\\resnet_model.h5")

# Load the image
image_path = "C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\test\\a2.jpg"
image = cv2.imread(image_path)

# Preprocess the image (resize, normalize, etc.)
# Example resizing to target size (100x100) and normalizing pixel values
target_size = (100, 100)
preprocessed_image = cv2.resize(image, target_size) / 255.0

# Expand dimensions to match the input shape expected by the model
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

# Predict the label
prediction = model.predict(preprocessed_image)

# Convert prediction to label
predicted_label_index = np.argmax(prediction)
 
confidence_score = prediction[0][predicted_label_index]

# Interpret the prediction
#class_labels = ["class_0", "class_1", "class_2", ...]  # Define your class labels
predicted_class_label = index_to_label.get(predicted_label_index)

print("Predicted Label:", predicted_label_index)
print("Predicted Label:", predicted_class_label)
print("Confidence Score:", confidence_score)
print("========================================================")

print(index_to_label)

print("========================================================")

1/1 [==============================] - 0s 91ms/step
Predicted Label: 6
Predicted Label: None
Confidence Score: 0.6362315
{'Aishwarya_Rai': 0, 'Akshay_Kumar': 1, 'Alia_Bhatt': 2, 'Asin': 3, 'Barack_Obama': 4, 'Brad_Pitt': 5, 'Deepika_Padukone': 6, 'Donald_Trump': 7, 'George_W_Bush': 8, 'Govinda': 9, 'Latha NV': 10, 'Meenal': 11, 'Rani_Mukerji': 12, 'Salman_Khan': 13, 'Vladimir_Putin': 14}


In [16]:
import os
import cv2
import numpy as np
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
import pickle

class_labels_file = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/trained_models/class_labels.pkl"

# Load class labels
with open(class_labels_file, "rb") as f:
    index_to_label = pickle.load(f)

# Load ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Add GlobalAveragePooling2D layer
x = GlobalAveragePooling2D()(base_model.output)

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Load the weights of the trained model
model.load_weights("C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\trained_models\\resnet_model.h5")

# Load the image
image_path = "C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\test\\a2.jpg"
image = cv2.imread(image_path)

# Preprocess the image
preprocessed_image = cv2.resize(image, (100, 100))
preprocessed_image = preprocess_input(preprocessed_image)

# Expand dimensions to match the input shape expected by the model
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

# Predict the features
features = model.predict(preprocessed_image)

# Use the features to make a prediction with your classifier
# Your classification code goes here...

# You can print the features for debugging if needed
print("Features:", features)


ValueError: Layer count mismatch when loading weights from file. Model expected 106 layers, found 4 saved layers.

In [18]:
import os
import cv2
import numpy as np
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
import pickle

class_labels_file = "C:/Users/Learning/udemy/Hackathon/ultimate/dataset/trained_models/class_labels.pkl"

# Load class labels
with open(class_labels_file, "rb") as f:
    index_to_label = pickle.load(f)

# Load ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Add GlobalAveragePooling2D layer
x = GlobalAveragePooling2D()(base_model.output)

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Load the weights of the trained model
model.load_weights("C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\trained_models\\resnet_model.h5")

# Load the image
image_path = "C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\test\\a2.jpg"
image = cv2.imread(image_path)

# Load the pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\trained_models\\haarcascade_frontalface_default.xml')

 
# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Process each detected face
for (x, y, w, h) in faces:
    # Draw a rectangle around the detected face
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the image with the square around the face
    cv2.imshow("Face Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Crop the face from the image
    face = image[y:y+h, x:x+w]

    # Save the cropped face image
    cv2.imwrite("C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\test\\cropped_face.jpg", face)

# Perform validation or further processing on the cropped face image
image = cv2.imread("C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\test\\cropped_face.jpg")
# Preprocess the image
preprocessed_image = cv2.resize(image, (100, 100))
preprocessed_image = preprocess_input(preprocessed_image)

# Expand dimensions to match the input shape expected by the model
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

# Predict the features
features = model.predict(preprocessed_image)

# Use the features to make a prediction with your classifier
# Your classification code goes here...

# You can print the features for debugging if needed
print("Features:", features)

ValueError: Layer count mismatch when loading weights from file. Model expected 106 layers, found 4 saved layers.